In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import re

### Read data from CSV 

In [2]:
# read the crime rates
df_cr = pd.read_csv('Crime_Rate_Per_State.csv', encoding='utf-8')
# read the income rates
df_ir = pd.read_csv('Household_Income.csv', encoding='utf-8')

In [3]:
df_cr.head(5)

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Alabama,4.86,4.38,4.45,4.29,4.27,4.33,4.25,4.49,4.53,4.50,3.84,4.20,4.50,4.31,4.27
1,Alaska,5.67,5.90,5.65,5.98,6.32,6.32,6.88,6.61,6.52,6.33,6.35,6.10,6.04,6.39,6.36
2,Arizona,5.32,5.40,5.55,5.13,5.04,5.12,5.43,5.18,4.86,4.27,4.14,4.14,4.29,4.16,4.00
3,Arkansas,4.45,4.52,4.25,4.56,5.02,5.28,5.53,5.37,5.13,5.16,5.04,4.82,4.70,4.63,4.80
4,California,6.22,6.15,5.95,5.80,5.28,5.26,5.34,5.24,5.04,4.73,4.40,4.11,4.24,4.03,3.96


In [4]:
df_ir.head(5)

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Alabama,3.54,3.52,3.76,3.73,3.66,3.72,3.80,4.22,4.45,4.00,4.09,4.26,4.35,4.73,4.23
1,Alaska,5.28,5.74,5.28,5.18,5.51,5.59,5.64,6.30,6.40,6.16,5.78,5.74,6.36,7.25,6.76
2,Arizona,3.98,4.27,3.97,4.12,4.38,4.52,4.67,4.72,4.69,4.57,4.69,4.86,4.70,5.26,4.93
3,Arkansas,2.97,3.33,3.24,3.20,3.50,3.67,3.71,4.08,3.96,3.65,3.86,4.13,3.90,3.94,4.49
4,California,4.68,4.73,4.74,4.93,4.92,5.18,5.53,5.57,5.70,5.61,5.43,5.34,5.70,6.08,6.05


In [5]:
df_cr.columns

Index(['State', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014'],
      dtype='object')

In [6]:
df_cr= df_cr.iloc[:,0:21]
df_cr.head()

,State,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Alabama,4.86,4.38,4.45,4.29,4.27,4.33,4.25,4.49,4.53,4.50,3.84,4.20,4.50,4.31,4.27
1,Alaska,5.67,5.90,5.65,5.98,6.32,6.32,6.88,6.61,6.52,6.33,6.35,6.10,6.04,6.39,6.36
2,Arizona,5.32,5.40,5.55,5.13,5.04,5.12,5.43,5.18,4.86,4.27,4.14,4.14,4.29,4.16,4.00
3,Arkansas,4.45,4.52,4.25,4.56,5.02,5.28,5.53,5.37,5.13,5.16,5.04,4.82,4.70,4.63,4.80
4,California,6.22,6.15,5.95,5.80,5.28,5.26,5.34,5.24,5.04,4.73,4.40,4.11,4.24,4.03,3.96


In [7]:
# convert the float to strings for crime rates
df_cr.columns = ['State']+['Y_'+str(col) for col in df_cr.columns if col != 'State']
df_cr.columns

Index(['State', 'Y_2000', 'Y_2001', 'Y_2002', 'Y_2003', 'Y_2004', 'Y_2005',
       'Y_2006', 'Y_2007', 'Y_2008', 'Y_2009', 'Y_2010', 'Y_2011', 'Y_2012',
       'Y_2013', 'Y_2014'],
      dtype='object')

In [8]:
# convert the float to strings for income rates
df_ir.columns = ['State']+['Y_'+str(col) for col in df_ir.columns if col != 'State']
df_ir.columns

Index(['State', 'Y_2000', 'Y_2001', 'Y_2002', 'Y_2003', 'Y_2004', 'Y_2005',
       'Y_2006', 'Y_2007', 'Y_2008', 'Y_2009', 'Y_2010', 'Y_2011', 'Y_2012',
       'Y_2013', 'Y_2014'],
      dtype='object')

### Checking for missing datapoints

In [9]:
missing_data_rows = df_cr[df_cr.isnull().any(1)]
missing_data_rows

,State,Y_2000,Y_2001,Y_2002,Y_2003,Y_2004,Y_2005,Y_2006,Y_2007,Y_2008,Y_2009,Y_2010,Y_2011,Y_2012,Y_2013,Y_2014


In [10]:
df_cr.head()

,State,Y_2000,Y_2001,Y_2002,Y_2003,Y_2004,Y_2005,Y_2006,Y_2007,Y_2008,Y_2009,Y_2010,Y_2011,Y_2012,Y_2013,Y_2014
0,Alabama,4.86,4.38,4.45,4.29,4.27,4.33,4.25,4.49,4.53,4.50,3.84,4.20,4.50,4.31,4.27
1,Alaska,5.67,5.90,5.65,5.98,6.32,6.32,6.88,6.61,6.52,6.33,6.35,6.10,6.04,6.39,6.36
2,Arizona,5.32,5.40,5.55,5.13,5.04,5.12,5.43,5.18,4.86,4.27,4.14,4.14,4.29,4.16,4.00
3,Arkansas,4.45,4.52,4.25,4.56,5.02,5.28,5.53,5.37,5.13,5.16,5.04,4.82,4.70,4.63,4.80
4,California,6.22,6.15,5.95,5.80,5.28,5.26,5.34,5.24,5.04,4.73,4.40,4.11,4.24,4.03,3.96


In [11]:
df_cr.columns

Index(['State', 'Y_2000', 'Y_2001', 'Y_2002', 'Y_2003', 'Y_2004', 'Y_2005',
       'Y_2006', 'Y_2007', 'Y_2008', 'Y_2009', 'Y_2010', 'Y_2011', 'Y_2012',
       'Y_2013', 'Y_2014'],
      dtype='object')

### Save to the sqlite database

In [12]:
engine = create_engine('sqlite:///../db/crime-rate.sqlite', echo=False)

In [13]:
# save the crime data into sqlite
df_cr.to_sql('crime_rates', con=engine, if_exists='replace')

In [14]:
# save the income data into sqlite
df_ir.to_sql('income_rates', con=engine, if_exists='replace')

In [15]:
engine.execute("SELECT * FROM crime_rates").fetchall()

[(0, 'Alabama', 4.86, 4.38, 4.45, 4.29, 4.27, 4.33, 4.25, 4.49, 4.53, 4.5, 3.84, 4.2, 4.5, 4.31, 4.27),
 (1, 'Alaska', 5.67, 5.9, 5.65, 5.98, 6.32, 6.32, 6.88, 6.61, 6.52, 6.33, 6.35, 6.1, 6.04, 6.39, 6.36),
 (2, 'Arizona', 5.32, 5.4, 5.55, 5.13, 5.04, 5.12, 5.43, 5.18, 4.86, 4.27, 4.14, 4.14, 4.29, 4.16, 4.0),
 (3, 'Arkansas', 4.45, 4.52, 4.25, 4.56, 5.02, 5.28, 5.53, 5.37, 5.13, 5.16, 5.04, 4.82, 4.7, 4.63, 4.8),
 (4, 'California', 6.22, 6.15, 5.95, 5.8, 5.28, 5.26, 5.34, 5.24, 5.04, 4.73, 4.4, 4.11, 4.24, 4.03, 3.96),
 (5, 'Colorado', 3.34, 3.5, 3.53, 3.47, 3.72, 3.97, 3.95, 3.52, 3.54, 3.39, 3.24, 3.14, 3.07, 3.05, 3.09),
 (6, 'Connecticut', 3.25, 3.35, 3.13, 3.17, 2.89, 2.73, 3.0, 3.01, 3.07, 3.01, 2.82, 2.76, 2.84, 2.62, 2.37),
 (7, 'Delaware', 6.84, 6.11, 6.0, 6.75, 6.15, 6.33, 7.01, 7.05, 7.09, 6.45, 6.23, 5.66, 5.51, 5.01, 4.89),
 (8, 'District of Columbia', 15.08, 16.02, 16.38, 16.25, 13.69, 13.8, 15.08, 14.15, 14.38, 13.49, 13.27, 12.01, 12.42, 12.96, 12.44),
 (9, 'Florida',

In [16]:
engine.execute("SELECT * FROM income_rates").fetchall()

[(0, 'Alabama', 3.54, 3.52, 3.76, 3.73, 3.66, 3.72, 3.8, 4.22, 4.45, 4.0, 4.09, 4.26, 4.35, 4.73, 4.23),
 (1, 'Alaska', 5.28, 5.74, 5.28, 5.18, 5.51, 5.59, 5.64, 6.3, 6.4, 6.16, 5.78, 5.74, 6.36, 7.25, 6.76),
 (2, 'Arizona', 3.98, 4.27, 3.97, 4.12, 4.38, 4.52, 4.67, 4.72, 4.69, 4.57, 4.69, 4.86, 4.7, 5.26, 4.93),
 (3, 'Arkansas', 2.97, 3.33, 3.24, 3.2, 3.5, 3.67, 3.71, 4.08, 3.96, 3.65, 3.86, 4.13, 3.9, 3.94, 4.49),
 (4, 'California', 4.68, 4.73, 4.74, 4.93, 4.92, 5.18, 5.53, 5.57, 5.7, 5.61, 5.43, 5.34, 5.7, 6.08, 6.05),
 (5, 'Colorado', 4.82, 4.94, 4.83, 4.99, 5.09, 5.04, 5.57, 6.11, 6.09, 5.59, 6.02, 5.86, 5.73, 6.79, 6.09),
 (6, 'Connecticut', 5.02, 5.33, 5.34, 5.5, 5.51, 5.68, 6.24, 6.41, 6.47, 6.49, 6.6, 6.54, 6.42, 6.93, 7.02),
 (7, 'Delaware', 5.04, 4.96, 4.97, 4.9, 4.8, 5.12, 5.24, 5.46, 5.07, 5.21, 5.52, 5.47, 4.9, 5.41, 5.75),
 (8, 'D.C.', 4.12, 4.12, 3.91, 4.5, 4.35, 4.5, 4.85, 5.08, 5.56, 5.31, 5.69, 5.53, 6.52, 6.01, 6.83),
 (9, 'Florida', 3.89, 3.64, 3.8, 3.9, 4.05, 4.3,

In [17]:
# Convert the index to primiary key
def get_create_table_string(tablename, connection):
    sql = """
    select * from sqlite_master where name = "{}" and type = "table"
    """.format(tablename) 
    result = connection.execute(sql)

    create_table_string = result.fetchmany()[0][4]
    return create_table_string

def add_pk_to_create_table_string(create_table_string, colname):
    regex = "(\n.+{}[^,]+)(,)".format(colname)
    return re.sub(regex, "\\1 PRIMARY KEY,",  create_table_string, count=1)

def add_pk_to_sqlite_table(tablename, index_column, connection):
    cts = get_create_table_string(tablename, connection)
    cts = add_pk_to_create_table_string(cts, index_column)
    template = """
    BEGIN TRANSACTION;
        ALTER TABLE {tablename} RENAME TO {tablename}_old_;

        {cts};

        INSERT INTO {tablename} SELECT * FROM {tablename}_old_;

        DROP TABLE {tablename}_old_;

    COMMIT TRANSACTION;
    """

    create_and_drop_sql = template.format(tablename = tablename, cts = cts)
    connection.executescript(create_and_drop_sql)

In [18]:
con = sqlite3.connect("../db/crime-rate.sqlite")

In [19]:
add_pk_to_sqlite_table('crime_rates', 'index', con)

In [20]:
add_pk_to_sqlite_table('income_rates', 'index', con)